In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import sys
import msgpack as mp
import msgpack_numpy as mpn
sys.path.append(os.path.join(os.getcwd(), ".."))
from support.generate_ar_data import *
from support.pd_support import *
from support.calculations_support import *
from support.mecanum_calculations import *
from support.ar_calculations import *

In [4]:
_parent_folder = "multi_cam_nov_14"
_folder_name = "sk16_15_sideways_fast"
_calib_filename = "calibration_15fps_quad"

_base_pth = os.path.dirname(os.getcwd())
_base_pth = os.path.join(_base_pth, "recording_programs", "test_data")

_pth = os.path.join(_base_pth, _parent_folder, _folder_name)
_calib_pth = os.path.join(_base_pth, _parent_folder, _calib_filename)

#calibration path
_webcam_calib_pth = os.path.join(_calib_pth, "webcam_calibration.msgpack")

#open the calibration files
with open(_webcam_calib_pth, "rb") as f:
    webcam_calib = mp.Unpacker(f, object_hook=mpn.decode)
    _temp = next(webcam_calib)
    _webcam_cam_mat = _temp[0]
    _webcam_dist = _temp[1]
_webcam_video_pth = os.path.join(_pth, "webcam_color.msgpack")
print("camera matrix", _webcam_cam_mat)
print("distortion", _webcam_dist)

camera matrix [[868.26070009   0.         372.25985418]
 [  0.         874.44598128 238.35209445]
 [  0.           0.           1.        ]]
distortion [[ 7.07394532e-02 -3.42419375e-02 -1.86205766e-03  8.20734788e-04
  -4.82051694e+00]]


In [5]:
def calculate_rotmat_from_3markers(corners, ids, camera_matrix, dist_coeffs, marker_length = 0.05):

    rotation_vector, translation_vector, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_length, camera_matrix, dist_coeffs)

    ids = list(ids)

    z_inx = ids.index(8)
    org_inx = ids.index(2)
    x_inx = ids.index(1)

    zvec = translation_vector[z_inx][0]
    zvec = np.reshape(zvec, (3, 1))
    org = translation_vector[org_inx][0] 
    org = np.reshape(org, (3, 1))
    xvec = translation_vector[x_inx][0]
    xvec = np.reshape(xvec, (3, 1))
    rotMat = calculate_rotmat(xvec, zvec, org)

    t_zvec = zvec - org 
    t_xvec = xvec - org

    print(rotMat.T@t_xvec) 
    print(rotMat.T@t_zvec)

    return rotMat, org

In [119]:
def detect_aruco(img):
    """
    Detects the lframe from the image

    Idx: 6 - zvector
    Idx: 9 - orgin
    Idx: 10 - xvector
    """
    aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_ARUCO_ORIGINAL)
    arucoParams = aruco.DetectorParameters_create()
    arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_CONTOUR   
    arucoParams.cornerRefinementWinSize = 5
    arucoParams.cornerRefinementMinAccuracy = 0.01
    arucoParams.cornerRefinementMaxIterations = 1000
    arucoParams.adaptiveThreshWinSizeMin = 10
    arucoParams.adaptiveThreshWinSizeMax = 50
    arucoParams.adaptiveThreshWinSizeStep = 10
    # arucoParams.adaptiveThreshConstant = 7
    # arucoParams.minMarkerPerimeterRate = 0.03
    # arucoParams.maxMarkerPerimeterRate = 4
    arucoParams.polygonalApproxAccuracyRate = 0.03
    # arucoParams.minCornerDistanceRate = 0.05
    # arucoParams.minDistanceToBorder = 3
    # arucoParams.minMarkerDistanceRate = 0.05
    arucoParams.aprilTagQuadSigma = 0.8
    corners, ids, rejectedImgPoints = aruco.detectMarkers(img, aruco_dict, parameters=arucoParams)

    return corners, ids, rejectedImgPoints

In [88]:
# open image file from msgpack
"""L frame orthogonalization"""
with open(os.path.join(_calib_pth, "webcam_frame.msgpack"), "rb") as f:
    webcam_video = mp.Unpacker(f, object_hook=mpn.decode)

    for i in webcam_video:
        _webcam_img = i
        break

In [89]:
corners, ids, _ = detect_aruco(_webcam_img)
# print(corners)
w_rotmat, w_org = calculate_rotmat_from_3markers(corners, ids, marker_length=0.05, camera_matrix=_webcam_cam_mat, dist_coeffs=_webcam_dist)

[[0.15652663]
 [0.        ]
 [0.        ]]
[[-1.70042205e-02]
 [-3.46944695e-18]
 [ 2.02218154e-01]]


In [120]:
"""open video from msgpack"""
with open(_webcam_video_pth, "rb") as f:
    webcam_video = mp.Unpacker(f, object_hook=mpn.decode)
    counter = 0
    for i in webcam_video:
        _webcam_img = i
        corners, ids, _ = detect_aruco(_webcam_img)
        if ids is not None:
            counter += 1
print(counter)
        

448
